## Load data from google sheet 
https://docs.google.com/spreadsheets/d/1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o/edit#gid=732292548

In [3]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('./key/questioniso-0a60ce8c39c1.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

sheet = client.open_by_key('1ZSLM2gc6GBWs1fvvNwzA0D7jusRWv2xPVqOVoXcYJ3o')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.worksheet('news')

data = sheet_instance.get_all_records()
print('data', data[:1])

data [{'id': 1, 'title': 'Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên giảm liên tiếp', 'content': 'Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\

Đưa dữ liệu vào dataframe.

In [4]:
df = pd.DataFrame()
title = []
content = []
date = []
stock_price = []
status = []

for values in data:
    title.append(values['title'])
    content.append(values['content'])
    date.append(values['date'])
    stock_price.append(values['stock_price'])
    status.append(values['status'])

df['title'] = title
df['content'] = content
df['date'] = date
df['stock_price'] = stock_price
df['status'] = status

df[0:5]

,title,content,date,stock_price,status
0,"Dow Jones tăng hơn 300 điểm, chấm dứt 5 phiên ...",Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3)...,2023/02/07,100.0,1.0
1,Vàng thế giới lùi nhẹ khi lợi suất trái phiếu ...,Giá vàng giảm vào ngày thứ Ba (14/3) khi lợi s...,2023/02/08,98.0,-0.5
2,Xét xử vụ DongABank: 'Lạnh gáy' với khoản vay ...,Sau khi bị cáo Phùng Ngọc Khánh (cựu Chủ tịch ...,2023/02/09,95.0,-1.0
3,Cựu tổng giám đốc Ngân hàng Đông Á Trần Phương...,Cùng các đồng phạm gây thiệt hại tổng số tiền ...,2023/02/10,93.0,-1.0
4,Nóng: Ngân hàng Nhà nước giảm lãi suất điều hành,Ngân hàng Nhà nước (NHNN) vừa ban hành các quy...,2023/02/11,94.0,1.0


Thực hiện clean text, cần apply nhiều thứ hơn.

In [5]:
import pandas as pd
from datetime import date, timedelta
import re
from underthesea import word_tokenize, sent_tokenize
from gensim import corpora, models

def clean_text(text):
    text = str(text).lower() # lowercase text
    return text

df = df[df['stock_price'] != '']

documents = df['content'].values.tolist()
print('documents:', documents[0:1])
print('len:', len(documents))

documents: ['Chứng khoán Mỹ khởi sắc vào ngày thứ Ba (14/3), khi nhà đầu tư đặt cược vào việc nguy cơ lan truyền sau vụ phá sản của các ngân hàng Silicon Valley và Signature đã được ngăn chặn.\n\n\n\nKết thúc phiên giao dịch ngày thứ Ba, chỉ số Dow Jones tăng 336.26 điểm (tương đương 1.06%) lên 32,155.40 điểm, chấm dứt chuỗi 5 phiên lao dốc liên tiếp. Chỉ số S&P 500 tiến 1.65% lên 3,919.29 điểm. Chỉ số Nasdaq Composite cộng 2.14% lên 11,428.15 điểm.\n\nSự nhiệt tình mua vào các cổ phiếu ngân hàng của nhà đầu tư đã giảm đi phần nào vào buổi chiều. Tuy nhiên, nhiều cổ phiếu vẫn ghi nhận đà tăng, đánh dấu bước đảo chiều sau 2 phiên giảm sâu khi nhà đầu tư ngày càng tin tưởng rằng những ngân hàng đó sẽ không chịu chung số phận như ngân hàng Silicon Valley và Signature. Các nhà quản lý cho biết vào ngày 12/3 rằng họ đã xây dựng một kế hoạch để can thiệp cho tất cả những người gửi tiền ở 2 ngân hàng này.\n\nChứng chỉ quỹ SPDR S&P Regional Banking ETF tiến 2%, phục hồi phần nào sau đà lao dốc

In [6]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

"""
    Create list document + list token.
"""
def cv_texts_to_tokens(texts):
    docs_tokened = []
    documents_cleaned = []
    for doc in texts:
        words = word_tokenize(str(doc).lower(), format="text").split(' ')
        words = [word.strip(".,?!") for word in words if word.isalpha()]
        docs_tokened.append(words)
        documents_cleaned.append(' '.join(words))
    return docs_tokened, documents_cleaned

docs_tokened, documents_cleaned = cv_texts_to_tokens(documents)

In [7]:
import gensim
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

w2v_model = gensim.models.KeyedVectors.load_word2vec_format('../model/wiki.vi.model.bin', binary=True)

In [8]:
print(docs_tokened)

[['mỹ', 'vào', 'ngày', 'thứ', 'ba', 'khi', 'vào', 'việc', 'sau', 'vụ', 'của', 'các', 'silicon', 'valley', 'và', 'signature', 'đã', 'được', 'phiên', 'ngày', 'thứ', 'ba', 'dow', 'jones', 'tăng', 'điểm', 'lên', 'điểm', 'chuỗi', 'phiên', 'lao', 'dốc', 's', 'p', 'tiến', 'lên', 'điểm', 'nasdaq', 'composite', 'cộng', 'lên', 'điểm', 'sự', 'mua', 'vào', 'các', 'của', 'đã', 'giảm', 'đi', 'vào', 'buổi', 'chiều', 'nhiều', 'vẫn', 'đà', 'tăng', 'bước', 'đảo', 'chiều', 'sau', 'phiên', 'giảm', 'sâu', 'khi', 'rằng', 'những', 'đó', 'sẽ', 'không', 'chịu', 'chung', 'như', 'silicon', 'valley', 'và', 'signature', 'các', 'nhà', 'cho', 'biết', 'vào', 'ngày', 'rằng', 'họ', 'đã', 'một', 'để', 'cho', 'những', 'người', 'gửi', 'tiền', 'ở', 'này', 'quỹ', 'spdr', 's', 'p', 'regional', 'banking', 'etf', 'tiến', 'sau', 'đà', 'lao', 'dốc', 'trong', 'phiên', 'trước', 'đó', 'first', 'republic', 'bank', 'gần', 'sau', 'khi', 'gần', 'vào', 'ngày', 'keycorp', 'vọt', 'gần', 'trong', 'đợt', 'nhẹ', 'sau', 'khi', 'trượt', 'dốc',

### Tính trung bình word2vec để tạo ra mảng training chỉ 2 chiều:
(dựa theo: https://github.com/PradipNichite/Youtube-Tutorials/blob/main/Yotutube_WordVectors.ipynb)

In [9]:

pdTrain = pd.DataFrame({'tokens': docs_tokened})

def sent_vec(sent):
    vector_size = w2v_model.vector_size
    wv_res = np.zeros(vector_size)
    ctr = 1
    for w in sent:
        if w in w2v_model:
            ctr += 1
            wv_res += w2v_model[w]
    wv_res = wv_res/ctr
    return wv_res
pdTrain['vec'] = pdTrain['tokens'].apply(sent_vec)
print(pdTrain.head())

                                              tokens  \
0  [mỹ, vào, ngày, thứ, ba, khi, vào, việc, sau, ...   
1  [giá, vàng, giảm, vào, ngày, thứ, ba, khi, mỹ,...   
2  [sau, khi, phùng, ngọc, khánh, cựu, hđqt, m, c...   
3  [cùng, các, gây, tiền, trên, tỉ, đồng, cựu, tr...   
4  [nhnn, vừa, các, giảm, từ, điểm, qua, còn, nhi...   

                                                 vec  
0  [0.4521556866843076, 0.36772873731374384, -0.9...  
1  [0.3843028273714601, 0.5310362149381412, -0.91...  
2  [0.09241913974457357, 0.3054196947547671, -0.8...  
3  [0.2170247682347419, 0.3187902290997966, -0.67...  
4  [0.2316580660791205, 0.7523304409403729, -0.98...  


In [10]:
y_train = df['status'].values.tolist()
statusNp = np.asarray(df['status'].values.tolist())
statusNp = np.expand_dims(statusNp, axis = 1)
print(statusNp.shape, statusNp, y_train)

(22, 1) [[ 1. ]
 [-0.5]
 [-1. ]
 [-1. ]
 [ 1. ]
 [-1. ]
 [-1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [ 1. ]
 [-1. ]
 [ 0.9]
 [-1. ]
 [-1. ]
 [-1. ]
 [ 0.5]
 [ 0.1]
 [ 0.4]
 [ 0.2]
 [-0.3]
 [-0.2]] [1.0, -0.5, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 0.9, -1.0, -1.0, -1.0, 0.5, 0.1, 0.4, 0.2, -0.3, -0.2]


## Version dùng mean numpy

In [11]:
print(w2v_model['ăn'])
print('m' in w2v_model)

[-4.0891525e-01  2.6084104e+00 -5.3833354e-01 -5.9752214e-01
  1.8656381e+00  6.6219044e-01 -3.7117357e+00 -4.5574969e-01
 -1.0396051e-01 -1.9559094e+00  9.7429109e-01 -4.0667385e-01
 -1.0615259e+00 -2.1646779e+00  9.3465680e-01 -1.4586806e+00
  5.8061433e-01 -3.7961490e+00  3.0390272e+00 -2.1457682e+00
 -5.7892412e-01  1.7761465e+00 -9.1266108e-01 -7.8270189e-02
  1.8554451e+00  1.5198513e+00 -6.2319130e-01 -2.2093023e-01
 -1.8296558e+00 -2.0173174e-01  1.8952109e+00 -6.9549829e-01
  1.8698134e-01  1.3218739e+00  2.4016383e+00  6.6352658e-02
 -2.7717566e-01 -2.4175718e+00 -1.1903220e+00 -7.9305309e-01
  1.3936094e+00  2.0585716e+00  2.4315950e-01 -8.1384414e-01
 -1.9550422e+00  2.2965547e-01 -1.5387051e-01 -1.1004342e+00
 -2.4111086e-01  1.3018471e-01 -3.8814726e-01 -9.9034458e-02
  1.6107775e+00  6.1631495e-01  1.9548521e+00  6.5761530e-01
 -1.9960139e+00  1.9396085e+00 -1.9843059e+00  8.8569456e-01
  6.8909150e-01 -1.4907720e+00 -8.0561894e-01 -1.4252144e-01
  8.3218837e-01 -7.37897

In [12]:
import numpy as np

def vectorize(sentence):
    words = word_tokenize(str(sentence).lower(), format="text").split(' ')
    words = [word.strip(".,?!") for word in words if word.isalpha()]
    words_vecs = [w2v_model[word] for word in words if word in w2v_model]
    
    if len(words_vecs) == 0:
        return np.zeros(400)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in documents])
print(X_train.shape)

(22, 400)


In [13]:
from sklearn.svm import SVR
clf_1 = SVR(kernel='linear')
clf_1.fit(X_train, y_train)


SVR(kernel='linear')

In [17]:
test_data = [
    "Công ty Lọc hóa dầu Bình Sơn (BSR) lên kế hoạch bảo dưỡng lớn cho nhà máy lọc dầu Dung Quất, đề ra mức lãi giảm gần 90%.",
    "Hiệu suất của chỉ số S&P 500 hiện chỉ tập trung vào một vài cổ phiếu lớn. Bảy trong số những cổ phiếu lớn nhất, ",
    "Frédéric Leroux, giám đốc bộ phận tài sản chéo tại Carmignac, cho biết: “Các cổ phiếu công nghệ lớn S&P hiện nay cũng giống như các công ty dầu mỏ trong quá khứ, hay Nifty 50 vào những năm 1960”. Ông cho biết “cơn sốt đầu tư” đã đẩy giá cổ phiếu của một số ít công ty tăng trưởng nhanh, như IBM, Kodak và Xerox, lên cao hơn trước khi giảm mạnh. “Đó là một vấn đề cứ bị lặp đi lặp lại”.",
]

X_test = np.array([vectorize(sentence) for sentence in test_data])
print(X_test.shape)
# print('X:\n', X_test)

result = clf_1.predict(X_test)
print('result:\n', result)

(3, 400)
result:
 [-2.71518331  1.83254513 -0.7313482 ]
